<a href="https://colab.research.google.com/github/macken1/DSB-TCT/blob/master/BERT%E6%97%A5%E6%9C%AC%E8%AA%9E_%EF%BC%88encode_plus%E3%83%90%E3%83%BC%E3%82%B8%E3%83%A7%E3%83%B3%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BERTを使って文をベクトルで表現してみる**

In [0]:
## 以下、Mecabを入れとかないとTokenizerでエラーとなるのでインストールしておく
## 乾研のBERTではmecab-ipadic-2.7.0-20070801との要件になっているが、最新版をインストール。影響は不明
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

## 1. 形態素解析（単語に分割する）ツールのMeCabをインストールする

Google colaboratory環境に、MeCab関連のツール（MeCab単体＋辞書、MeCabをpythonから実行）などをインストールします。

In [0]:
# coding: utf-8
# 念のため、形態素解析ができるかチェック

import MeCab

m = MeCab.Tagger("Owakati")
print(m.parse("私はKARAが大好きだった。コンサートに行ったころが懐かしい。"))
print(m.parse("コロナウイルスとプロスペクト理論に悩む今日この頃"))
print(m.parse("すもももももももものうち"))

Colabでグラフ表示する際に文字化けしないように、必要なライブラリをインストールします

In [0]:
#　以下はmatplotlibでの日本語表示用のライブラリ
!pip install japanize_matplotlib
import matplotlib.pyplot as plt
import japanize_matplotlib 
import seaborn as sns

sns.set(font="IPAexGothic")

In [0]:
!pip install transformers
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import transformers as ppb

# 東北大学乾研が作成した学習済みの日本語BERTモデルの形態素解析モデルを搭載します。
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')

参照用にBERTモデル内の語彙（形態素）をvocabulary.txtに書き出します。<br>左のウインドウからvocabulary.txtをダブルクリックすると内容が確認できます。

In [0]:
# write all tokens in vocabulary.txt, which you can find on the left-hand side
with open("vocabulary.txt", 'w') as f:
    
    # For each token...
    for token in tokenizer.vocab.keys():
        
        # Write it out and escape any unicode characters.            
        f.write(token + '\n')


辞書に含まれている形態素の長さ（＝文字カウント）について分布をみてみます

In [0]:
import numpy as np

sns.set(style='darkgrid',font="IPAexGothic")

# Increase the plot size and font size.
sns.set(font_scale=1.5,font="IPAexGothic")
plt.rcParams["figure.figsize"] = (10,5)

# Measure the length of every token in the vocab.
token_lengths = [len(token) for token in tokenizer.vocab.keys()]

# Plot the number of tokens of each length.
sns.countplot(token_lengths)
plt.title('形態素の長さの分布')
plt.xlabel('形態素の長さ')
plt.ylabel('形態素数')

print('最大の形態素長:', max(token_lengths))

文のベクトルを使って似た文を抽出してみましょう
せっかくなので、たくさんの文章からベクトル化を使って意味の似た文章を抽出してみましょう。  
まず、データを用意します。  
京都大学の黒橋・河原研究室のホームページ(※1)から「Textual Entailment 評価データ」をダウンロードしてデータファイルを作ります。
※１　https://bit.ly/2sXN2er

In [0]:
!wget http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/rte/entail_evaluation_set.xml -P /content/

import xml.etree.ElementTree as ET
import codecs

# 出力するファイルを指定
f1 = codecs.open('T_plain.txt', 'w', 'utf-8')

# XMLファイルからT1とT2の正例を抽出して、ファイルに出力
corpus = ET.parse('entail_evaluation_set.xml')
root = corpus.getroot()

for child in root:
  grandchild_text = {}
  entail_tag = child.get('label')
  for grandchild in child.getchildren():
    grandchild_text[grandchild.tag] = grandchild.text
  if entail_tag == '◎' or entail_tag == '〇':
    f1.write(grandchild_text['t1']+"\n")
    f1.write(grandchild_text['t2']+"\n") 

どんな文が含まれているのか、10個ほどサンプリングして表示してみます

In [0]:
import pandas as pd
df = pd.read_table("/content/T_plain.txt", header=None)
df.sample(10)

文例をいったんリスト化します。

In [0]:
sentences = df[0].values
print(sentences)

実際に、日本語の文章がどのように形態素、さらにBERTの分析用のIDに変換されるのかを見てみてましょう

In [0]:
# Print the original sentence.
print('原文: ', sentences[0])

# Print the sentence split into tokens.
print('形態素: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('形態素に対応するID番号化: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

文をBERTで分析する際、入力側の形態素数（≒単語数）を固定する必要があります。

「Textual Entailment 評価データ」の形態素の数を見てみましょう。

※CLS、SEPなどの特別な形態素も含めてカウントしています

In [0]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('最長の形態素数は⇒ ', max_len)

BERTモデルを読み込みます。

In [0]:
from transformers import BertJapaneseTokenizer, BertModel

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
# どうもpytorchには学習モードと推論モードがあるらしい。以下は推論モードに切り替え
model.eval()

文例を一気に形態素の順にID化します。ここでは同時に短い文例は32形態素になるよう０で残りを埋めています。

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print('attention_masks:', attention_masks[0])

ID化した文例をBERTモデルを使って形態素ごとにベクトル化し、アウトプットのテンソルの次元を確認します

In [0]:
# GPU上で走らせるためには、関連するものをすべてGPU上に配置
input_ids = input_ids.to('cuda')
attention_masks = attention_masks.to('cuda')
model.to('cuda')

# いよいよ推論させる、というか各形態素に対応するベクトルを計算
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
# 以下で素のアウトプット
last_hidden_states = outputs[0]
last_hidden_states.shape
# last_hidden_states.type

吐き出したテンソルのうち、CLSに相当するテンソルのみ切り出します

In [0]:
# NumPy配列ndarrayの要素や部分配列（行・列など）は[2, 3, 1, ...]のように各次元の位置や範囲をカンマ区切りで指定。[:,0,:]でCLSに対応するはず
last_hidden_states[:,0,:].shape

例に似た文章をコサイン類似度を使って似ている順番に抽出します。まず、形態素×ID化

In [0]:
# 例文と類似する文を探してみる
example = '猫がニャーニャーとうるさくて眠れない。'

# encode_plusでpaddingまでできる。超便利。ptでPyTorch、tfでTensorflow、何もつけないと普通にリストを返す
example_ids = tokenizer.encode_plus(example, max_length=32, pad_to_max_length=True, return_tensors='pt')
print(example_ids)
type(example_ids)
print(example_ids["input_ids"]) #辞書から必要な部分を参照

そのうえで、BERTを使ってベクトル化します。

In [0]:
# GPU上で走らせるためには、関連するものをすべてGPU上に配置
input_ids_example = example_ids["input_ids"].to('cuda')
attention_masks_example = example_ids["attention_mask"].to('cuda')
model.to('cuda')

# いよいよ推論させる、というか各形態素に対応するベクトルを計算
with torch.no_grad():
    example_outputs = model(input_ids_example, attention_mask=attention_masks_example)
# 以下で素のアウトプット
last_hidden_states_example = example_outputs[0]
last_hidden_states_example[:,0,:].shape
# last_hidden_states.type

コサイン類似度をすべて文例と計算し、上位20例を表示します

In [0]:
corpus = codecs.open('/content/T_plain.txt', 'r', 'utf-8')
corpussimdic = {}

# コサイン類似度を計算する
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# 例文のベクトル
example_vec = last_hidden_states_example[:,0,:].to('cpu').detach().numpy().copy()

# コーパス文のベクトル
corpus_vec = last_hidden_states[:,0,:].to('cpu').detach().numpy().copy() 
  
print()
print(example, "<=>")

#
# コーパスの文とのコサイン類似度を求める
#
i = 0
for sentence in corpus:

  corpussimdic[sentence.rstrip('\n')] = cos_sim(example_vec, corpus_vec[i,:])
  i += 1


# valueで降順にソートしてトップ20の類似文を表示する
count = 0
for k, v in sorted(corpussimdic.items(), key=lambda x: -x[1]):
    print(str(v) + ": " + str(k)) 
    count += 1 
    if count == 20:
      break
